# super を使って、スーパークラスを初期化する

In [1]:
# サブクラスからスーパークラスを初期化する古いやり方
class MyBaseClass:
  def __init__(self, value):
    self.value = value

class MyChildClass(MyBaseClass):
  def __init__(self):
    MyBaseClass.__init__(self, 5)

In [6]:
# 古い方式は、単純な階層では問題ない
# クラスが、多重継承（一般的には避ける）によって影響を受けているとき、
# スーパークラスの __init__ メソッドを直接呼び出すと、
# 予期せぬ振る舞いに遭遇することがある
class TimesTwo:
  def __init__(self):
    self.value *= 2

class PlusFive:
  def __init__(self):
    self.value += 5

In [7]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
  def __init__(self, value):
    MyBaseClass.__init__(self, value)
    TimesTwo.__init__(self)
    PlusFive.__init__(self)

In [8]:
foo = OneWay(5)
print('First ordering is (5 * 2) + 5 =', foo.value)

First ordering is (5 * 2) + 5 = 15


In [ ]:
# TimesTwo と PlusFive の順序がことなるクラスを定義しても、まだ結果は同じ
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
  def __init__(self, value):
    MyBaseClass.__init__(self, value)
    TimesTwo.__init__(self)
    PlusFive.__init__(self)

In [12]:
bar = AnotherWay(5)
print('Second ordering still is', bar.value)

Second ordering still is 15


In [13]:
class TimesSeven(MyBaseClass):
  def __init__(self, value):
    MyBaseClass.__init__(self, value)
    self.value * 7

class PlusNine(MyBaseClass):
  def __init__(self, value):
    MyBaseClass.__init__(self, value)
    self.value += 9

In [18]:
class ThisWay(TimesSeven, PlusNine):
  def __init__(self, value):
    TimesSeven.__init__(self, value)
    PlusNine.__init__(self, value)

foo = ThisWay(5)
print('Should be (5 * 7) + 9 = 44 but is', foo.value)

Should be (5 * 7) + 9 = 44 but is 14


2番目のスーパークラスのコンストラクタ PlusNine.\__init__ の呼び出しでは、MyBaseClass._\_init__ が2回目に呼び出されたところで、self.value が 5 にリセットされる。
ので、TimesSeven._\_init__ コンストラクタの効果が無視され、self.value の計算は 5 + 9 = 14 となってしまう

In [27]:
# この問題を解決するために、Python は組み込み関数 super と標準メソッド解決順序 MRO を用意
# super は、ダイアモンド階層の共通スーパークラスが一度しか実行されないことを保証する
# MRO は、C3 線形化と呼ばれるアルゴリズムにしたがってスーパークラスの初期化順を定義する
class TimesSevenCorrect(MyBaseClass):
  def __init__(self, value):
    super().__init__(value)
    self.value *= 7

class PlusNineCorrect(MyBaseClass):
  def __init__(self, value):
    super().__init__(value)
    self.value += 9

In [31]:
class GoodWay(TimesSevenCorrect, PlusNineCorrect):
  def __init__(self, value):
    super().__init__(value)

foo = GoodWay(5)
print('Should be 7 * (5 + 9) = 98 and is', foo.value)

Should be 7 * (5 + 9) = 98 and is 98


この順序は奇妙に思えるかもしれない。つまり、TimesSevenCorrect.\__init__ が最初に実行されるべきでは？
と思うかもしれないが、答えは No である

In [28]:
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print(mro_str)

<class '__main__.GoodWay'>
<class '__main__.TimesSevenCorrect'>
<class '__main__.PlusNineCorrect'>
<class '__main__.MyBaseClass'>
<class 'object'>


GoodWay(5) を呼び出すと、

1. TimesSevenCorrect.\__init__
1. PlusNineCorrect.\__init__
1. MyBaseClass.\__init__

を呼び出す

ダイアモンドの頂点に達すると、初期化メソッドのすべては、その \__init__ 関数が呼ばれたのと逆順で作業をする

In [32]:
# 継承する順序を返れば答えは変わる
class GoodWay(PlusNineCorrect, TimesSevenCorrect):
  def __init__(self, value):
    super().__init__(value)

foo = GoodWay(5)
print('Should be (7 * 5) + 9 = 44 and is', foo.value)

Should be (7 * 5) + 9 = 44 and is 44


In [ ]:
# 補足: super にはオプションの引数がある
# 以下3つは等価
class ExplicitTrisect(MyBaseClass):
  def __init__(self, value):
    super(ExplicitTrisect, self).__init__(value)
    self.value /= 3

class AutomaticTrisect(MyBaseClass):
  def __init__(self, value):
    super(__class__, self).__init__(value)
    self.value /= 3

class ImpricitTrisect(MyBaseClass):
  def __init__(self, value):
    super().__init__(value)
    self.value /= 3

assert ExplicitTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImpricitTrisect(9).value == 3

#### 補足

> super に引数を渡さないといけないのは、スーパークラスの実装の特定の機能にサブクラスからアクセスしなければいけない場合だけです

について

これは主に「親のメソッドを明示的に取り出してラップする」場合を指している。
以下のような、メソッドを取得して保存・再利用するような高度なケース

In [34]:
class LoggingDict(dict):
    def __setitem__(self, key, value):
        print(f"Setting {key} = {value}")
        super().__setitem__(key, value)

class UppercaseDict(LoggingDict):
    def __setitem__(self, key, value):
        # super() に引数を明示する（UppercaseDict, self）
        # これで LoggingDict.__setitem__ を取得して呼び出せる
        log_setitem = super(UppercaseDict, self).__setitem__
        log_setitem(key.upper(), value)

In [35]:
d = UppercaseDict()
d['foo'] = 'bar'
print(d)

Setting FOO = bar
{'FOO': 'bar'}


UppercaseDict は LoggingDict を継承している

ここで super(UppercaseDict, self).\__setitem__ を使って、LoggingDict の \__setitem__ を明示的に取得して、それをラップして使う

このように、スーパークラスのメソッドを変数に格納して後で呼び出すときは、super() に引数を渡す必要がある。

なぜなら、super() を引数なしで呼ぶと「その場ですぐに呼び出す」ケースが前提になっているからである。